# RL Test Bench

In [1]:
import sys
pwd = "P:/dsv/DAMII/Practical Project/rl_stock_agents"
sys.path.append(pwd)

import os
import cleandata
import numpy as np
import matplotlib.pyplot as plt
from agents import Discrete_QtabAgent, Discrete_Random_Agent, Discrete_Buy_Hold_Agent
from agentperform import agent_stock_performance
import logging

# Output Location
main_path = "P:/dsv/DAMII/Practical Project/rl_stock_agents/test_bench"
test_dir = '/_test/'
save_path = main_path + test_dir

## Check if the directory exists; if not, create it
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Logger Setup (Do not change)
logging.basicConfig(level=logging.INFO, filemode='w')
## Test Logger
test_logger = logging.getLogger('Test_logger')
test_handler = logging.FileHandler(save_path +'/test.log')
test_handler.setLevel(logging.INFO)
test_logger.addHandler(test_handler)

## History Logger
history_logger = logging.getLogger('History_logger')
history_handler = logging.FileHandler(save_path + '/history.log')
history_handler.setLevel(logging.INFO)
history_logger.addHandler(history_handler)

## Model Logger
model_compare_logger = logging.getLogger('Model_compare_logger')
model_compare_handler = logging.FileHandler(save_path + '/model_compare.log')
model_compare_handler.setLevel(logging.INFO)
model_compare_logger.addHandler(model_compare_handler)

## Workbench Configuration

In [2]:
# Input Data Location, File Name, Stock name for labels
import_path = "P:/dsv/DAMII/Practical Project/rl_stock_agents/input_data"

stock_list =["amzn_daily.csv", "appl_daily.csv","tsla_daily.csv","f_daily.csv",
             "pfe_daily.csv", "coke_daily.csv","brk_daily.csv","nee_daily.csv",
             "jnj_daily.csv","pg_daily.csv"]

stock_name_list = ['AMZN','APPL','TSLA','F','PFE','COKE','BRK','NEE','JNJ','PG']

# Configuration for Grpahic Output
save_graphic_option = True
output_graphic_option = False


# Common Model Parameters 
training_range = [[0,1000]]
test_range = [[1000,1250]]
epochs = 5
num_of_tests = 2
metrics = ['n_trades','win_percentage','cumulative_return','sortino','max_drawdown','sharpe']
## See agentperform.py -> results dictionary for options
## current setting "n_trades" has its ranks based on min (manual change in
## prob._valuate.py line 92. Need to create feature to avoid this manual change)
rank_order = ['min','max','max','max','min','max']


# Agent Declariations 

agent_classes = {'Qtab_[2,2,2,2,2]': Discrete_QtabAgent,
                 'Qtab_[10,10,10,10,10]': Discrete_QtabAgent,
                 'Qtab_[100,100,100,100,100]': Discrete_QtabAgent,
                 'Qtab_[100,100,100,400,400]': Discrete_QtabAgent,
                 'Random_Agent': Discrete_Random_Agent,
                 'Buy_Hold_Agent':Discrete_Buy_Hold_Agent}

agent_list = list(agent_classes.keys())

no_train_agents = ['Random_Agent','Buy_Hold_Agent']  

agent_params = {
    'Qtab_[2,2,2,2,2]':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'num_training_episodes': epochs,
        'testing_idxs': test_range,
        'epsilon':  0.1,
        'alpha':  0.1,
        'gamma':  0.9
    },

        'Qtab_[10,10,10,10,10]':{
        'bins_per_feature': [10,10,10,10,10],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'num_training_episodes': epochs,
        'testing_idxs': test_range,
        'epsilon':  0.1,
        'alpha':  0.1,
        'gamma':  0.9
    },

        'Qtab_[100,100,100,100,100]':{
            'bins_per_feature': [100,100,100,100,100],
            'bin_padding': 0.25,
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.1,
            'gamma':  0.9
    },

        'Qtab_[100,100,100,400,400]':{
        'bins_per_feature': [100,100,100,400,400],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'num_training_episodes': epochs,
        'testing_idxs': test_range,
        'epsilon':  0.1,
        'alpha':  0.1,
        'gamma':  0.9    
        },


    'Random_Agent':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'testing_idxs': test_range,
    },
        'Buy_Hold_Agent':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'testing_idxs': test_range,
    }
    
    
    }
    

stock_list =["amzn_daily.csv", "appl_daily.csv","tsla_daily.csv","f_daily.csv",
             "pfe_daily.csv", "coke_daily.csv","brk_daily.csv","nee_daily.csv",
             "jnj_daily.csv","pg_daily.csv"]

stock_name_list = ['AMZN','APPL','TSLA','F','PFE','COKE','BRK','NEE','JNJ','PG']









## Code for Analysis, shouldn't have to be touched

In [3]:
results = {}

for file_name, stock_name in zip(stock_list,stock_name_list):
    history_logger.info(stock_name)
    results[stock_name] = {}
    # Import File
    df_ohlcv = cleandata.NASDAQ_csv_input(file_name,import_path)
    
    for agent_name, agent_class in agent_classes.items():
        
        selected_agent = agent_class(df_ohlcv,
                                **agent_params[agent_name])

        # Output Status    
        update_text = f'{agent_name}:{stock_name} - Initialized'    
        print(update_text)

        # history Log  
        history_logger.info(update_text)
        
        results[stock_name][agent_name] = {}
                                           
        # Training and Testing Pairs  
        for (train_start_idx, train_end_idx), (test_start_idx, test_end_idx) \
            in zip(selected_agent.training_range, selected_agent.test_range):
            
            selected_agent.reset() # Wipe out any internal values
                 
            if agent_name not in no_train_agents:  ### Exclude if agent does not require training
                train_key = f'[{train_start_idx}:{train_end_idx})' 
                graph_train_key = 'TRN'+ train_key
                ### Output Status
                update_text = f'{agent_name}:{stock_name} - Training: {train_key}'    
                print(update_text)
                
                ### Train Agent
                selected_agent.train(train_start_idx,
                                     train_end_idx,
                                     initial_epsilon = 0.75, 
                                     final_epsilon = 0.1)
                ### History Logger 
                history_logger.info(update_text)
                history_logger.info(selected_agent.env.step_info[-1:])
            
            ## Testing Section 

            # Marker for Output Status, Debugging, and Return
            test_key = f'[{test_start_idx}:{test_end_idx})'
            
            # For Graph Title 
            graph_test_key = 'TST'+ test_key


            # History Log                                   
            history_logger.info(test_key)
            
            # Initializing Results Dictionary for Test Keys
            results[stock_name][agent_name][test_key] = []

            # Combine Graph Title
            if agent_name in no_train_agents:
                graph_label = f'{agent_name}$_{{{graph_test_key}}}$'
            else:
                graph_label = f'{agent_name}$_{{{graph_train_key},{graph_test_key}}}$'

            for idx in range(1, num_of_tests + 1):

                # Combine Graph Title
                if agent_name in no_train_agents:
                
                    graph_image_file = f'{agent_name}-{stock_name}-TST_{test_start_idx}_{test_end_idx}-{idx}.png'
                else:
                            
                    graph_image_file = f'{agent_name}-{stock_name}-TRN_{train_start_idx}_{train_end_idx}-TST_{test_start_idx}_{test_end_idx}-{idx}.png'
                
                image_file_path = save_path + graph_image_file

                # Output Status    
                update_text = f'{agent_name}:{stock_name} -  \
                    Testing: [{test_start_idx}:{test_end_idx}) - Test #: {idx}'    
                print(update_text)

                # History Logging
                history_logger.info(update_text)    
                                    
                selected_agent.test(test_start_idx,test_end_idx)

                trades_list = [entry['Action'] for entry in \
                               selected_agent.env.step_info]
        
                trade_seq = np.array(trades_list)

                # History Logging
                history_logger.info(selected_agent.env.step_info[-5:])

                # History Logging
                history_logger.info(trade_seq)


                test_result = agent_stock_performance(df_ohlcv['close']\
                        .iloc[test_start_idx:test_end_idx].to_numpy(),
                        trade_seq,stock_name,graph_label, 
                        display_graph=output_graphic_option,
                        save_graphic=save_graphic_option, 
                        path_file=image_file_path)
                
                # Update Results Dic
                result_details = {metric:test_result[metric] for metric in metrics}
                results[stock_name][agent_name][test_key].append([trade_seq,result_details])

# Write to Test Logger
test_logger.info(results)


INFO:History_logger:AMZN
INFO:History_logger:Qtab_[2,2,2,2,2]:AMZN - Initialized
INFO:History_logger:Qtab_[2,2,2,2,2]:AMZN - Training: [0:1000)
INFO:History_logger:[{'Step': 1000, 'idx': 999, 'Portfolio Value': 921630.5077439994, 'Cash': 921630.5077439994, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 58.957, 'Last Commission Cost': 0, 'Total Commision Cost': 452670.39569100004, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('H',), 'Action': 'H'}]
INFO:History_logger:[1000:1250)
INFO:History_logger:Qtab_[2,2,2,2,2]:AMZN -                      Testing: [1000:1250) - Test #: 1
INFO:History_logger:[{'Step': 246, 'idx': 1245, 'Portfolio Value': 1114311.8182525, 'Cash': 1114311.8182525, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 82.0515, 'Last Commission Cost': 0, 'Total Commision Cost': 137418.1095475, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('H', 'B'), 'Action': 'B'}, {'Step': 247, 'idx': 1246, 'Portfolio Value': 1119883.1151025, 'Cash': 5623.7451024998445, 

Qtab_[2,2,2,2,2]:AMZN - Initialized
Qtab_[2,2,2,2,2]:AMZN - Training: [0:1000)
Qtab_[2,2,2,2,2]:AMZN -                      Testing: [1000:1250) - Test #: 1


INFO:History_logger:Qtab_[2,2,2,2,2]:AMZN -                      Testing: [1000:1250) - Test #: 2
INFO:History_logger:[{'Step': 246, 'idx': 1245, 'Portfolio Value': 1026888.6073699999, 'Cash': 1026888.6073699999, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 82.0515, 'Last Commission Cost': 0, 'Total Commision Cost': 112586.97714000002, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('H', 'B'), 'Action': 'H'}, {'Step': 247, 'idx': 1246, 'Portfolio Value': 1026888.6073699999, 'Cash': 1026888.6073699999, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 82.162, 'Last Commission Cost': 0, 'Total Commision Cost': 112586.97714000002, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('H', 'B'), 'Action': 'H'}, {'Step': 248, 'idx': 1247, 'Portfolio Value': 1026888.6073699999, 'Cash': 1026888.6073699999, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 83.177, 'Last Commission Cost': 0, 'Total Commision Cost': 112586.97714000002, 'Input': [0, 0, 0, 0, 0], 'Available Action

Qtab_[2,2,2,2,2]:AMZN -                      Testing: [1000:1250) - Test #: 2


INFO:History_logger:Qtab_[10,10,10,10,10]:AMZN - Initialized
INFO:History_logger:Qtab_[10,10,10,10,10]:AMZN - Training: [0:1000)
INFO:History_logger:[{'Step': 1000, 'idx': 999, 'Portfolio Value': 961129.1384625003, 'Cash': 961129.1384625003, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 58.957, 'Last Commission Cost': 0, 'Total Commision Cost': 451462.3697414998, 'Input': [2, 2, 2, 2, 1], 'Available Actions': ('H',), 'Action': 'H'}]
INFO:History_logger:[1000:1250)
INFO:History_logger:Qtab_[10,10,10,10,10]:AMZN -                      Testing: [1000:1250) - Test #: 1
INFO:History_logger:[{'Step': 246, 'idx': 1245, 'Portfolio Value': 986438.0315949997, 'Cash': 986438.0315949997, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 82.0515, 'Last Commission Cost': 0, 'Total Commision Cost': 113442.35901500002, 'Input': [3, 3, 3, 3, 2], 'Available Actions': ('H', 'B'), 'Action': 'B'}, {'Step': 247, 'idx': 1246, 'Portfolio Value': 991370.1472599998, 'Cash': 4947.014259999734, 'S

...

INFO:History_logger:Buy_Hold_Agent:PG - Initialized
INFO:History_logger:[1000:1250)
INFO:History_logger:Buy_Hold_Agent:PG -                      Testing: [1000:1250) - Test #: 1
INFO:History_logger:[{'Step': 246, 'idx': 1245, 'Portfolio Value': 1132437.7, 'Cash': 5009.949999999953, 'Stock Value': 1134500.85, 'Stock Holdings': 12195, 'Stock Price': 93.03, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.95, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 247, 'idx': 1246, 'Portfolio Value': 1139510.8, 'Cash': 5009.949999999953, 'Stock Value': 1145232.45, 'Stock Holdings': 12195, 'Stock Price': 93.91, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.95, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 248, 'idx': 1247, 'Portfolio Value': 1150242.4, 'Cash': 5009.949999999953, 'Stock Value': 1146695.85, 'Stock Holdings': 12195, 'Stock Price': 94.03, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.9

Buy_Hold_Agent:PG - Initialized
Buy_Hold_Agent:PG -                      Testing: [1000:1250) - Test #: 1


c:\Users\beckm\Anaconda3\lib\site-packages\quantstats\stats.py:349: RuntimeWarning: divide by zero encountered in scalar divide
  res = returns.mean() / downside
INFO:History_logger:Buy_Hold_Agent:PG -                      Testing: [1000:1250) - Test #: 2
INFO:History_logger:[{'Step': 246, 'idx': 1245, 'Portfolio Value': 1011202.9602000001, 'Cash': 5069.610200000112, 'Stock Value': 1012445.49, 'Stock Holdings': 10883, 'Stock Price': 93.03, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.6502, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 247, 'idx': 1246, 'Portfolio Value': 1017515.1002000001, 'Cash': 5069.610200000112, 'Stock Value': 1022022.5299999999, 'Stock Holdings': 10883, 'Stock Price': 93.91, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.6502, 'Input': [0, 0, 0, 0, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 248, 'idx': 1247, 'Portfolio Value': 1027092.1402, 'Cash': 5069.610200000112, 'Stock Value': 10233

Buy_Hold_Agent:PG -                      Testing: [1000:1250) - Test #: 2


INFO:Test_logger:{'AMZN': {'Qtab_[2,2,2,2,2]': {'[1000:1250)': [[array(['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'S', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B',
       'S', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'S', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'H', 'S', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'S', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'S', 'H',
       'H', 'H', 'B', 'S', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'B', 'S',

# Aggreating Test Bench Data

In [4]:
import pandas as pd

test_keys = [f"[{interval[0]}:{interval[1]})" for interval in test_range]
aggerate_results = {}
for agent in agent_list:
    aggerate_results[agent] = {}
    for stock in stock_name_list:
        aggerate_results[agent][stock] = {}
        for test in test_keys:
            aggerate_results[agent][stock][test] = {}
            values = np.empty((0,len(metrics)))
            for test_num in range(num_of_tests):
                values_array = [results[stock][agent][test][test_num][1][key] for key in metrics]
                current_values = np.array(values_array)
                values = np.vstack((values,current_values))

                means_for_metrics = np.mean(values, axis=0)
                std_for_metrics = np.std(values, axis=0)

            
            
            for idx,metric in enumerate(metrics):
                aggerate_results[agent][stock][test][metric] = (means_for_metrics[idx],std_for_metrics[idx])

display(aggerate_results)

summarized_aggerate_results = {}

for metric in metrics:
    model_list = []
    dataset_name = []
    scores = []
    for agent in aggerate_results.keys():
        
        model_list.append(agent)
        score_list = []
        for stock in aggerate_results[agent].keys():
            for test in aggerate_results[agent][stock].keys():
                run_name = stock + "-" + test
                if run_name not in dataset_name:
                    dataset_name.append(run_name)
                score = aggerate_results[agent][stock][test][metric][0]
                score_list.append(np.round(score,2))
        scores.append(score_list)

    score_array = np.array(scores).T

    df = pd.DataFrame(score_array,columns=model_list)
    df['dataset'] = dataset_name

    column_order = ['dataset'] + [col for col in df.columns if col != 'dataset']
    df = df[column_order]

    model_means = list(zip(model_list,df[model_list].mean()))

    summarized_aggerate_results[metric] = (df, model_means)


display(summarized_aggerate_results)



{'Qtab_[2,2,2,2,2]': {'AMZN': {'[1000:1250)': {'n_trades': (13.0, 2.0),
    'win_percentage': (48.18181818181819, 11.818181818181817),
    'cumulative_return': (1.049338661887551, 0.022325128203074196),
    'sortino': (5.232609954243054, 1.453332900954719),
    'max_drawdown': (-3.865626913196124, 0.14250150128065076),
    'sharpe': (2.9008791808983467, 0.854761544853841)}},
  'APPL': {'[1000:1250)': {'n_trades': (9.5, 1.5),
    'win_percentage': (53.97727272727273, 8.522727272727273),
    'cumulative_return': (1.008666836982427, 0.017418635689231465),
    'sortino': (5.876832247268687, 6.781906864847363),
    'max_drawdown': (-3.5770565493474624, 2.3397926594841354),
    'sharpe': (2.35448421914607, 2.9933316959096405)}},
  'TSLA': {'[1000:1250)': {'n_trades': (13.0, 2.0),
    'win_percentage': (48.18181818181819, 11.818181818181817),
    'cumulative_return': (1.071528359703139, 0.08594613532443446),
    'sortino': (4.804426631632342, 5.764890509276044),
    'max_drawdown': (-8.323897

{'n_trades': (            dataset  Qtab_[2,2,2,2,2]  Qtab_[10,10,10,10,10]  \
  0  AMZN-[1000:1250)              13.0                   10.0   
  1  APPL-[1000:1250)               9.5                   11.5   
  2  TSLA-[1000:1250)              13.0                  117.5   
  3     F-[1000:1250)              12.5                   12.5   
  4   PFE-[1000:1250)              12.0                   11.5   
  5  COKE-[1000:1250)              13.5                   12.0   
  6   BRK-[1000:1250)              16.5                   12.5   
  7   NEE-[1000:1250)              12.5                   13.0   
  8   JNJ-[1000:1250)              17.0                   15.5   
  9    PG-[1000:1250)              13.0                   11.0   
  
     Qtab_[100,100,100,100,100]  Qtab_[100,100,100,400,400]  Random_Agent  \
  0                         9.0                         9.0          60.5   
  1                        11.5                        13.0          64.0   
  2                        1

# Significance

In [5]:
import prob_evaluate

for metric, rank_type in list(zip(metrics,rank_order)):
    test = prob_evaluate.generate_rank_array_from_dataframe(summarized_aggerate_results[metric][0],
                                                            model_list,equal_rank_behav="mean",rank_order=rank_type)
    
    
    model_compare_logger.info(f'\n\n\n-------{metric}--------\nFriedman Test:')
    model_compare_logger.info(summarized_aggerate_results[metric][0])
    model_compare_logger.info(test.T)
    stat, critical_f_value, reject_null_hypo = prob_evaluate.iman_davenport_test(test,0.95)

    model_compare_logger.info(f'f-score: {stat:.3f}\nf-critical: {critical_f_value:.3f}\nSignificant: {reject_null_hypo}')

    results1 = prob_evaluate.nemenyi_test(test,0.95,model_list)
    model_compare_logger.info(f'Nemenyi Test Resutls:\n{results1}')

INFO:Model_compare_logger:n_trades
Friedman Test:
INFO:Model_compare_logger:            dataset  Qtab_[2,2,2,2,2]  Qtab_[10,10,10,10,10]  \
0  AMZN-[1000:1250)              13.0                   10.0   
1  APPL-[1000:1250)               9.5                   11.5   
2  TSLA-[1000:1250)              13.0                  117.5   
3     F-[1000:1250)              12.5                   12.5   
4   PFE-[1000:1250)              12.0                   11.5   
5  COKE-[1000:1250)              13.5                   12.0   
6   BRK-[1000:1250)              16.5                   12.5   
7   NEE-[1000:1250)              12.5                   13.0   
8   JNJ-[1000:1250)              17.0                   15.5   
9    PG-[1000:1250)              13.0                   11.0   

   Qtab_[100,100,100,100,100]  Qtab_[100,100,100,400,400]  Random_Agent  \
0                         9.0                         9.0          60.5   
1                        11.5                        13.0          64